In [1]:
import yaml
from torch.utils.data import DataLoader
from DataLoader import (
    AfricanWildlifeDataset,
    bernoulli_noise_transform,
    gaussian_noise_transform,
)
from DenoisingAE import DenoisingAE, DenoisingAEV1, DenoisingAEV2
from DenoisingResnet import DenoisingResnet
import torch
import gc
import torchvision
import yaml
import gin
import os
from itertools import product
import shutil
import numpy as np
import matplotlib.image
import cv2

checkpoints = {
    "gaussian": {
        0.1: {
            "v0": "denoising_checkpoints/gaussian/0.1/v0/best.ckpt",
            "v1": "denoising_checkpoints/gaussian/0.1/v1/best.ckpt",
            "v2": "denoising_checkpoints/gaussian/0.1/v2/best.ckpt",
        },
        0.2: {
            "v0": "denoising_checkpoints/gaussian/0.2/v0/best.ckpt",
            "v1": "denoising_checkpoints/gaussian/0.2/v1/best.ckpt",
            "v2": "denoising_checkpoints/gaussian/0.2/v2/best.ckpt",
        },
    },
    "bernoulli": {
        0.1: {
            "v0": "denoising_checkpoints/bernoulli/0.1/v0/best.ckpt",
            "v1": "denoising_checkpoints/bernoulli/0.1/v1/best.ckpt",
            "v2": "denoising_checkpoints/bernoulli/0.1/v2/best.ckpt",
        },
        0.3: {
            "v0": "denoising_checkpoints/bernoulli/0.3/v0/best.ckpt",
            "v1": "denoising_checkpoints/bernoulli/0.3/v1/best.ckpt",
            "v2": "denoising_checkpoints/bernoulli/0.3/v2/best.ckpt",
        },
    },
}
torch.cuda.empty_cache()
gc.collect()

gin.parse_config_file("config.cfg")

ParsedConfigFileIncludesAndImports(filename='config.cfg', imports=[], includes=[])

In [2]:
from pathlib import Path
import os

y = None
with open(f"./wildlife.yaml") as stream:
    this_path = os.path.join(os.getcwd(), "aaa")
    try:
        y = yaml.safe_load(stream)
        print(y)
        y["datasets_dir"] = this_path
        print(y)
        # yaml.safe_dump(y, stream)
    except yaml.YAMLError as exc:
        print(exc)

# with open(f"{Path.home()}/.config/Ultralytics/settings.yaml", "w") as f:
#     yaml.dump(y, f)

{'path': '../datasets/wildlife', 'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'names': {0: 'buffalo', 1: 'elephant', 2: 'rhino', 3: 'zebra'}}
{'path': '../datasets/wildlife', 'train': 'train/images', 'val': 'valid/images', 'test': 'test/images', 'names': {0: 'buffalo', 1: 'elephant', 2: 'rhino', 3: 'zebra'}, 'datasets_dir': '/home/edoardo/aiproject/aaa'}


In [3]:
def copy_images(dataset, model, noise_type, noise):
    for index in range(len(dataset)):
        img, orig = dataset[index]
        shape = dataset.image_dimensions[index]
        original_name = dataset.list_dir[index]

        t = torchvision.transforms.Resize((shape[1], shape[2]))

        # Clean and returning to the original size
        cleanedv0 = t(model(img.to("cuda")).detach().cpu())
        # Transpose the axes to make it WHC and reconvert it to a [0,...,255] image
        normalized = np.transpose((cleanedv0 * 255).numpy(), [1, 2, 0]).astype(
            dtype=np.uint8
        )
        matplotlib.image.imsave(
            f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/{dataset.kind}/images/{original_name}",
            normalized,
        )


noises = {"gaussian": [0.1, 0.2], "bernoulli": [0.1, 0.3]}

for noise_type in ["gaussian", "bernoulli"]:
    for noise in noises[noise_type]:
        if noise_type == "gaussian":
            noise_transform = gaussian_noise_transform(0, noise, width=640)
        else:
            noise_transform = bernoulli_noise_transform(noise, width=640)
        modelv2 = DenoisingAEV2.load_from_checkpoint(
            checkpoints[noise_type][noise]["v2"]
        )
        modelv1 = DenoisingAEV1.load_from_checkpoint(
            checkpoints[noise_type][noise]["v1"]
        )
        modelv0 = DenoisingAE.load_from_checkpoint(checkpoints[noise_type][noise]["v0"])
        for model in [modelv0, modelv1, modelv2]:
            train_dataset = AfricanWildlifeDataset(
                kind="train", transform=noise_transform
            )
            val_dataset = AfricanWildlifeDataset(
                kind="valid", transform=noise_transform
            )
            test_dataset = AfricanWildlifeDataset(
                kind="test", transform=noise_transform
            )
            dataset_yaml = dict(
                {
                    "path": f"../datasets/wildlife_{noise_type}_{noise}_{model.kind}",
                    "train": "train/images",
                    "val": "valid/images",
                    "test": "test/images",
                    "names": {0: "buffalo", 1: "elephant", 2: "rhino", 3: "zebra"},
                }
            )
            with open(f"./wildlife_{noise_type}_{noise}_{model.kind}.yaml", "w") as f:
                yaml.dump(dataset_yaml, f)
            os.mkdir(f"datasets/wildlife_{noise_type}_{noise}_{model.kind}")

            os.mkdir(f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/train")
            os.mkdir(
                f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/train/images"
            )
            # we just copy the labels because we have kept the right image dimensions
            shutil.copytree(
                "datasets/wildlife/train/labels",
                f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/train/labels",
            )
            os.mkdir(f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/test")
            os.mkdir(f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/test/images")
            # we just copy the labels because we have kept the right image dimensions
            shutil.copytree(
                "datasets/wildlife/test/labels",
                f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/test/labels",
            )
            os.mkdir(f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/valid")
            os.mkdir(
                f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/valid/images"
            )
            # we just copy the labels because we have kept the right image dimensions
            shutil.copytree(
                "datasets/wildlife/valid/labels",
                f"datasets/wildlife_{noise_type}_{noise}_{model.kind}/valid/labels",
            )
            copy_images(train_dataset, model, noise_type, noise)
            copy_images(test_dataset, model, noise_type, noise)
            copy_images(val_dataset, model, noise_type, noise)

/home/edoardo/anaconda3/envs/aiproject/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
